In [1]:
pip install ijson

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 2.1 MB/s eta 0:00:00


In [18]:
pip install rank_bm25

In [10]:
!pip install hnswlib


  Using cached hnswlib-0.8.0.tar.gz (36 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2287619 sha256=bcec5c47b0448beee8ac776e612f74c272aa1a55f3016506cb11096a0d56aae5
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


In [12]:
import json
import ijson
import numpy as np
import nltk
from nltk.corpus import stopwords
#from krovetzstemmer import Stemmer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import BertTokenizer, BertModel
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import pandas as pd
import ast
import hnswlib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [19]:
from rank_bm25 import BM25Okapi

In [7]:
def get_top_k_aip(query,docs,categories,scores,k):
    res = sorted(list(zip(docs,categories,scores)),key = lambda x:x[2],reverse=True)
    aip = ""
    for i in range(min(k,len(scores))):
        ppep = 'the category for the article:"'+res[i][0]+'" is "'+res[i][1]+'", and '
        aip +=ppep
    aip = aip[:-6] + "."+query
    return aip

def preprocess(text,stopwords,stemmer):
    text = text.split()
    text = [text[i] for i in range(len(text)) if text[i] not in stopwords]
    text = [stemmer.stem(text[i]) for i in range(len(text))]
    return text
def get_data(file_path):
    all_data = []
    with open(file_path, 'r') as file:
        json_objects = ijson.items(file, 'item')
        for data in json_objects:
            all_data.append(data)
    f1 = open('all_data.txt','w+')
    f1.write(str(all_data))
    f1.close()
    articles = []
    for i,data in enumerate(all_data):
    ###Uncomment if using train data
        if i!=2112:
            articles.append(data['input'].split('article:')[-1])
        else:
            articles.append(data['input'].split('article:')[-2]+" "+data['input'].split('article:')[-1])
    return all_data,articles

def format_aips(aipfile,outputsfile):
    f = open(aipfile,"r")
    print(aipfile)
    aip = eval(f.read())
    aip = aip[:101]
    f2 = open(outputsfile)
    y_true = json.load(f2)
    y_true = y_true["golds"]
    y_true = [i['output'] for i in y_true]

    llm_data = pd.DataFrame({
        'text':aip,
        'label':y_true
    })
    op_name = str(aipfile).split('.txt')[0]+"_llm_output_100.csv"
    llm_data.to_csv(op_name)
    return

In [15]:
def hnsw(all_data,vectorizer):
    aip = []
    all_scores = []
    count=0
    for instance in all_data:
        query = instance['input']
        docs = [instance['profile'][i]['text'] for i in range(len(instance['profile']))]
        titles = [instance['profile'][i]['title'] for i in range(len(instance['profile']))]
        document_vectors = vectorizer.fit_transform(docs).toarray()
        dim = document_vectors.shape[1]
        num_elements = len(docs)
        index = hnswlib.Index(space='cosine', dim=dim)
        index.init_index(max_elements=num_elements, ef_construction=50, M=20)
        index.add_items(document_vectors)
        #decide how much ef
        index.set_ef(50)
        query_vector = vectorizer.transform([query]).toarray()[0]
        max_score_index, distances = index.knn_query(query_vector, k=num_elements)
        max_score_index = max_score_index.tolist()
        distances = distances.tolist()
        res = sorted(list(zip(max_score_index[0],distances[0])))
        scores = [1-res[i][1] for i in range(len(res))]
        all_scores.append(scores)
        title = titles[max_score_index[0][0]]
        op = '"'+title+'" is the title for "'+ docs[max_score_index[0][0]] +'".'+query
        aip.append(op)
        count+=1
        print(count)
    return aip,all_scores


In [11]:
def ensemble(all_data,models):
    n = len(all_data)
    aip = []
    txt = "the category for the article: "
    num_models = len(models)
    all_overall = []
    for i in range(n):
        query = all_data[i]['input']
        categories = [all_data[i]['profile'][j]['category'] for j in range(len(all_data[i]['profile']))]
        docs  = [all_data[i]['profile'][j]['text'] for j in range(len(all_data[i]['profile']))]
        ndocs = len(docs)
        overall_scores = [0 for i in range(ndocs)]
        for j in range(num_models):
            overall_scores = [overall_scores[k]+models[j][i][k] for k in range(ndocs)]
        overall_scores = [(1/num_models)*overall_scores[k] for k in range(ndocs)]
        aip.append(news.get_top_k_aip(query,docs,categories,overall_scores,5))
        all_overall.append(overall_scores)
        #max_score_index = overall_scores.index(max(overall_scores))
        #op = '"'+txt +'"'+docs[max_score_index]+'" is "'+categories[max_score_index]+'"".'+all_data[i]['input']
        #aip.append(op)
    return aip, all_overall

In [13]:
def bm25_model(all_data,articles):
    aip=[]
    all_scores = []
    #else:
        #articles.append(data['input'].split('article:')[-2]+" "+data['input'].split('article:')[-1])
    #print(len(articles),len(all_data))
    txt = "the category for the article: "
    for i in range(len(all_data)):
        query = articles[i]
        #query = all_data[i]['input']
        docs = [all_data[i]['profile'][j]['text'] for j in range(len(all_data[i]['profile'])) ]
        tokenized_docs = [doc.split() for doc in docs]
        #print("docs",docs)
        category = [all_data[i]['profile'][j]['category'] for j in range(len(all_data[i]['profile'])) ]
        bm25 = BM25Okapi(tokenized_docs)
        scores = bm25.get_scores(query.split())
        #min max normalization
        min_score = min(scores)
        max_score = max(scores)
        if max_score == min_score:
            scores = [1.0 for i in range(len(scores))]
        else:
            scores = [(score- min_score)/(max_score-min_score)  for score in scores]
        all_scores.append(scores)
        max_score_index = np.argmax(scores)
        op = '"'+txt +'"'+docs[max_score_index]+'" is "'+category[max_score_index]+'"".'+all_data[i]['input']
        aip.append(op)
    #print("IN BM25",len(aip))
    return aip,all_scores


In [17]:
def bert(all_data,tokenizer,model):
    aip = []
    all_scores=[]
    progress_bar = tqdm(range(len(all_data)))
    txt = "the category for the article: "
    for instance in all_data:
        query = instance['input']
        docs = [instance['profile'][i]['text'] for i in range(len(instance['profile']))]
        category =[instance['profile'][i]['category'] for i in range(len(instance['profile']))]
        document_tokens = tokenizer.batch_encode_plus(docs, padding=True, truncation=True, return_tensors='pt').to('cuda')
        query_tokens = tokenizer.encode_plus(query, padding=True, truncation=True, return_tensors='pt').to('cuda')
        document_embeddings = model(**document_tokens).last_hidden_state.mean(dim=1)  # Average pooling over tokens
        query_embedding = model(**query_tokens).last_hidden_state.mean(dim=1).squeeze()
        similarities = cosine_similarity(query_embedding.reshape(1, -1).detach().cpu().numpy(), document_embeddings.detach().cpu().numpy()).squeeze()
        progress_bar.update(1)
        ranking = sorted(enumerate(similarities), key=lambda x: x[1], reverse=True)
        #print(ranking)
        max_score_index = ranking[0][0]
        op = '"'+txt +'"'+docs[max_score_index]+'" is "'+category[max_score_index]+'"".'+instance['input']
        aip.append(op)
        ranking = sorted(ranking)
        scores = [ranking[i][1] for i in range(len(ranking))]
        #print("SCORES",ranking,scores)
        all_scores.append(scores)
    return aip, all_scores

In [14]:
def fwrite(output,filename):
    f = open(filename,'w+')
    f.write(str(output))
    f.close()
    return

In [ ]:
file_path = 'train_questions.json'
op_file_path = 'train_outputs.json'
all_data, articles = get_data(file_path)

In [ ]:
bm25_output,bm25_scores = bm25.bm25_model(all_data,articles)
fwrite(bm25_output,'bm_25_aip.txt')
fwrite(bm25_scores,'bm_25_scores.txt')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
bert_output,bert_scores = bert.bert(all_data,tokenizer,model)
model.to('cuda')
fwrite(bert_output,'bert/train/bert_aip_train.txt')
fwrite(bert_scores,'bert/train/bert_scores_train.txt')